In [1]:
import sys
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import nn

sys.path.append("../model")
from datamodule import TSDataModule
from model import LitLSTM, LSTMModel
from utils import evaluate

/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "../../data/input_data.csv"
num_time_steps = 6

batch_size = 32
learning_rate = 1e-3

num_features = 9
hidden_size = 16
num_lstm_layers = 1
bidirectional = False

ts_data_module = TSDataModule(
    path,
    num_time_steps=num_time_steps,
    batch_size=batch_size
)

lstm_net = LSTMModel(
    input_size=num_features,
    hidden_size=hidden_size,
    num_layers=num_lstm_layers,
    bidirectional=bidirectional,
)
loss_func = nn.MSELoss()
pl_net = LitLSTM(lstm_net, loss_func)

checkpoint_callback = ModelCheckpoint(
    save_top_k=3,
    monitor="val_loss",
    mode="min",
    filename="lstm-{epoch:04d}-{val_loss:.8f}",
)
trainer = pl.Trainer(callbacks=[checkpoint_callback])
trainer.fit(model=pl_net, datamodule=ts_data_module)

/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'lstm_net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['lstm_net'])`.
  rank_zero_warn(
/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'loss_func' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_func'])`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


(2534, 9) (634, 9) (793, 9)



  | Name      | Type      | Params
----------------------------------------
0 | lstm_net  | LSTMModel | 1.7 K 
1 | loss_func | MSELoss   | 0     
----------------------------------------
1.7 K     Trainable params
0         Non-trainable params
1.7 K     Total params
0.007     Total estimated model params size (MB)
2023-03-05 16:08:58.062680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 16:08:58.062740: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Sanity Checking: 0it [00:00, ?it/s]

/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/davidham/miniconda3/envs/tf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1: 100%|██████████| 99/99 [00:02<00:00, 43.81it/s, loss=0.0203, v_num=10]

In [ ]:
evaluate(pl_net, ts_data_module)

{'rmse': 1157.90667801857, 'R': 0.49537842925744624, 'mape': 30.632574}